### GraphRAG: Graph-Enhanced Retrieval-Augmented Generation

#### Overview

###### GraphRAG is an advanced question-answering system that combines the power of graph-based knowledge representation with retrieval-augmented generation. It processes input documents to create a rich knowledge graph, which is then used to enhance the retrieval and generation of answers to user queries. The system leverages natural language processing, machine learning, and graph theory to provide more accurate and contextually relevant responses.

#### Motivation
###### Traditional retrieval-augmented generation systems often struggle with maintaining context over long documents and making connections between related pieces of information. GraphRAG addresses these limitations by:

###### Representing knowledge as an interconnected graph, allowing for better preservation of relationships between concepts.
###### Enabling more intelligent traversal of information during the query process.
###### Providing a visual representation of how information is connected and accessed during the answering process.

#### Key Components
###### DocumentProcessor: Handles the initial processing of input documents, creating text chunks and embeddings.

###### KnowledgeGraph: Constructs a graph representation of the processed documents, where nodes represent text chunks and edges represent relationships between them.

###### QueryEngine: Manages the process of answering user queries by leveraging the knowledge graph and vector store.

###### Visualizer: Creates a visual representation of the graph and the traversal path taken to answer a query.

#### Method Details
###### Document Processing:

###### Input documents are split into manageable chunks.
###### Each chunk is embedded using a language model.
###### A vector store is created from these embeddings for efficient similarity search.

#### Knowledge Graph Construction:

###### Graph nodes are created for each text chunk.
###### Concepts are extracted from each chunk using a combination of NLP techniques and language models.
###### Extracted concepts are lemmatized to improve matching.
###### Edges are added between nodes based on semantic similarity and shared concepts.
###### Edge weights are calculated to represent the strength of relationships.

#### Query Processing:

###### The user query is embedded and used to retrieve relevant documents from the vector store.
###### A priority queue is initialized with the nodes corresponding to the most relevant documents.
###### The system employs a Dijkstra-like algorithm to traverse the knowledge graph:
###### Nodes are explored in order of their priority (strength of connection to the query).
###### For each explored node:
###### Its content is added to the context.
###### The system checks if the current context provides a complete answer.
###### If the answer is incomplete:
###### The node's concepts are processed and added to a set of visited concepts.
###### Neighboring nodes are explored, with their priorities updated based on edge weights.
###### Nodes are added to the priority queue if a stronger connection is found.
###### This process continues until a complete answer is found or the priority queue is exhausted.
###### If no complete answer is found after traversing the graph, the system generates a final answer using the accumulated context and a large language model.

#### Visualization:

###### The knowledge graph is visualized with nodes representing text chunks and edges representing relationships.
###### Edge colors indicate the strength of relationships (weights).
###### The traversal path taken to answer a query is highlighted with curved, dashed arrows.
###### Start and end nodes of the traversal are distinctly colored for easy identification.
###### Benefits of This Approach
###### Improved Context Awareness: By representing knowledge as a graph, the system can maintain better context and make connections across different parts of the input documents.

###### Enhanced Retrieval: The graph structure allows for more intelligent retrieval of information, going beyond simple keyword matching.

###### Explainable Results: The visualization of the graph and traversal path provides insight into how the system arrived at its answer, improving transparency and trust.

###### Flexible Knowledge Representation: The graph structure can easily incorporate new information and relationships as they become available.

###### Efficient Information Traversal: The weighted edges in the graph allow the system to prioritize the most relevant information pathways when answering queries.

###### Conclusion
###### GraphRAG represents a significant advancement in retrieval-augmented generation systems. By incorporating a graph-based knowledge representation and intelligent traversal mechanisms, it offers improved context awareness, more accurate retrieval, and enhanced explainability. The system's ability to visualize its decision-making process provides valuable insights into its operation, making it a powerful tool for both end-users and developers. As natural language processing and graph-based AI continue to evolve, systems like GraphRAG pave the way for more sophisticated and capable question-answering technologies.

#### Import relevant libraries 

In [ ]:
import networkx as nx
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.callbacks import get_openai_callback

from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import sys
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import List, Tuple, Dict
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
import spacy
import heapq

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import numpy as np

from spacy.cli import download
from spacy.lang.en import English

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variables
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)


### Define the document processor class